## L4 - 1: Regression Model with TF DenseFeatures

### Instructions
- Build regression model to predict resting blood pressure (trestbps field in the dataset) using Tensorflow DenseFeatures
- Please include the age and sex features and create a TF cross feature(https://www.tensorflow.org/api_docs/python/tf/feature_column/crossed_column) with them by binning the age field 
- Evaluate with common regression(MSE, MAE) and classification metrics(accuracy, F1, precision and recall across classes, AUC). No ROC or PR curve needed since this is a regression model that was converted to have a binary output and does not have the confidence in a given prediction.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, classification_report, roc_auc_score
#from https://archive.ics.uci.edu/ml/datasets/Heart+Disease
swiss_dataset_path = "./data/heart_disease_data/processed_swiss.csv"
swiss_df = pd.read_csv(swiss_dataset_path).replace('?', np.nan)

In [ ]:
column_list = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num_label']

In [ ]:
cleveland_df = pd.read_csv("./data/heart_disease_data/processed.cleveland.txt",  names=column_list).replace('?', np.nan)

In [ ]:
combined_heart_df = pd.concat([swiss_df, cleveland_df])

In [ ]:
len(combined_heart_df)

426

In [ ]:
# Review the data
combined_heart_df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num_label
0,32.0,1.0,1.0,95,0.0,NaN,0,127,0,.7,1,NaN,NaN,1
1,34.0,1.0,4.0,115,0.0,NaN,NaN,154,0,.2,1,NaN,NaN,1
2,35.0,1.0,4.0,NaN,0.0,NaN,0,130,1,NaN,NaN,NaN,7,3
3,36.0,1.0,4.0,110,0.0,NaN,0,125,1,1,2,NaN,6,1
4,38.0,0.0,4.0,105,0.0,NaN,0,166,0,2.8,1,NaN,NaN,2


In [ ]:
# It would appear that we have some na values to deal with
combined_heart_df.isna().sum()

age            0
sex            0
cp             0
trestbps       2
chol           0
fbs           75
restecg        1
thalach        1
exang          1
oldpeak        6
slope         17
ca           122
thal          54
num_label      0
dtype: int64

In [ ]:
# Based on the solution, it would appear that we're only trainin on specific features.
# Subset?
combined_heart_df = combined_heart_df[['sex',  'age', 'trestbps', 'thalach']]

In [ ]:
# The simple method is just to drop em'. Based on what I see here, this might be 
# severe...
print(combined_heart_df.shape)
clean_df = combined_heart_df.dropna()
print(clean_df.shape)

(426, 4)
(424, 4)


In [ ]:
clean_df.dtypes

sex         float64
age         float64
trestbps     object
thalach      object
dtype: object

In [ ]:
# Skipping ahead, it looks as though we're being encouraged to recast sex as a string
clean_df["sex"] = np.where(clean_df["sex"] == 1, "male", "female")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
clean_df.head()

,sex,age,trestbps,thalach
0,male,32.0,95,127
1,male,34.0,115,154
3,male,36.0,110,125
4,female,38.0,105,166
5,female,38.0,110,156


In [ ]:
# It's unclear to my why some of these features are reading as object. So, recast
clean_df['trestbps'] = clean_df['trestbps'].astype("float")
clean_df['thalach'] = clean_df['thalach'].astype("float")

clean_df.dtypes

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


sex          object
age         float64
trestbps    float64
thalach     float64
dtype: object

In [ ]:
# Define the train and test groups
training_data = clean_df.sample(frac=0.8)
test_data = clean_df.drop(training_data.index)

In [ ]:
#adapted from https://www.tensorflow.org/tutorials/structured_data/feature_columns
def df_to_dataset(df, predictor,  batch_size=32):
    df = df.copy()
    labels = df.pop(predictor)
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(batch_size)
    return ds

In [ ]:
# Convert to tensors
PREDICTOR_FIELD = 'trestbps'
batch_size = 128
train_ds = df_to_dataset(training_data, PREDICTOR_FIELD, batch_size=batch_size)
test_ds = df_to_dataset(test_data, PREDICTOR_FIELD, batch_size=batch_size)

In [ ]:
# Preprocess numerical features -- there is only one
tf_numeric_age_feature = tf.feature_column.numeric_column(key="age", 
                                                          default_value=0,
                                                         dtype=tf.float64)

In [ ]:
# Based on what I'm seeing, they want us to bucket age, too?
b_list = [ 0, 18, 25, 40, 55, 65, 80, 100]
#create TF bucket feature from numeric feature
tf_numeric_age_feature
tf_bucket_age_feature = tf.feature_column.bucketized_column(source_column=tf_numeric_age_feature, boundaries= b_list)

In [ ]:
# And then one-hot encode the gender column... Which, given cardinality, doesn't really 
# require a vocab.

# Here, save in memory as oppose to writtin to file

# I think it's also relevant to mention here that we're running this on the WHOLE file,
# not just the splits
gender_vocab = tf.feature_column.categorical_column_with_vocabulary_list(
      'sex', clean_df['sex'].unique())
gender_one_hot = tf.feature_column.indicator_column(gender_vocab)

In [ ]:
# Now, this is new: cross features...
# More here: https://www.tensorflow.org/api_docs/python/tf/feature_column/crossed_column

# A way to combine categorical features? Here, we combine the age and the gender features.
crossed_age_gender_feature = tf.feature_column.crossed_column([tf_bucket_age_feature, gender_vocab], hash_bucket_size=1000)
tf_crossed_age_gender_feature = tf.feature_column.indicator_column(crossed_age_gender_feature)

NameError: name 'tf' is not defined

In [ ]:
feature_columns = [ tf_crossed_age_gender_feature, tf_bucket_age_feature, gender_one_hot ]

# Create the densefeatures layer
dense_feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [ ]:
# Use same architecture as example
def build_model(dense_feature_layer):
  model = tf.keras.Sequential([
    dense_feature_layer,
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [ ]:
# Build the model
model = build_model(dense_feature_layer)

# Train
# No validation set b/c need to build separate TF dataset
EPOCHS = 2000
# Set to patience to 100 so it trains to end of epochs
early_stop = tf.keras.callbacks.EarlyStopping(monitor='mse', patience=10)     
history = model.fit(train_ds,   callbacks=[early_stop], epochs=EPOCHS,  verbose=1)

NameError: name 'dense_feature_layer' is not defined

In [ ]:
loss, mae, mse = model.evaluate(test_ds, verbose=2)
print("MAE:{}\nMSE:{}".format(mae, mse))

In [ ]:
test_labels = test_dataset[PREDICTOR_FIELD].values
test_predictions = model.predict(test_ds).flatten()

In [ ]:
model_pred_outputs = {
    "pred": test_predictions,
    "actual_value": test_labels,
}
model_output_df = pd.DataFrame(model_pred_output

In [ ]:
# Convert Regression Output to binary classification output
model_output_df.tail()

In [ ]:
#convert to binary prediction for Brier score - resting bps above 130 
def convert_to_binary(df, pred_field, actual_field):
    # score is the field name we will use for predictions and is what Aequitas uses
    df['score'] = df[pred_field].apply(lambda x: 1 if x>=130 else 0 )
    # label_value is the field name we will use for the truth value and is what Aequitas uses
    df['label_value'] = df[actual_field].apply(lambda x: 1 if x>=130 else 0)
    return df

In [ ]:
binary_df = convert_to_binary(model_output_df, 'pred', 'actual_value')
binary_df.head()

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, roc_auc_score

In [ ]:
y_true = binary_df['label_value'].values 
y_pred = binary_df['score'].values

In [ ]:
accuracy_score(y_true, y_pred)

In [ ]:
print(classification_report(y_true, y_pred))

In [ ]:
roc_auc_score(y_true, y_pred)